In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.driver.bindAddress","127.0.01") \
    .getOrCreate()

In [7]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2022-11-22 13:17:20--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:21d6:4200:b:20a5:b140:21, 2600:9000:21d6:5e00:b:20a5:b140:21, 2600:9000:21d6:6a00:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:21d6:4200:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet’

fhvhv_tripdata_2021 100%[===================>] 294,61M  48,9MB/s    in 6,5s    

2022-11-22 13:17:27 (45,3 MB/s) - ‘fhvhv_tripdata_2021-01.parquet’ saved [308924937/308924937]



In [8]:
!wc -l fhvhv_tripdata_2021-01.parquet

 1006794 fhvhv_tripdata_2021-01.parquet


In [32]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [13]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [14]:
!head -n 1001 fhvhv_tripdata_2021-01.parquet > head.parquet

In [18]:
import pandas as pd
import pyarrow.parquet as pq

In [20]:
df_pandas = pd.read_parquet('fhvhv_tripdata_2021-01.parquet')


In [21]:
df_pandas.dtypes

hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
request_datetime        datetime64[ns]
on_scene_datetime       datetime64[ns]
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int64
DOLocationID                     int64
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
dtype: object

In [22]:
spark.createDataFrame(df_pandas).schema

/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


KeyboardInterrupt: 

Integer - 4 bytes
Long - 8 bytes

In [25]:
from pyspark.sql import types

In [27]:
schema = types.StructType([
    types.StructField("hvfhs_license_num", types.StringType(), True),
    types.StructField("dispatching_base_num",  types.StringType(), True),
    types.StructField("originating_base_num",  types.StringType(), True),
    types.StructField("request_datetime",  types.TimestampType(), True),
    types.StructField("on_scene_datetime",  types.TimestampType(), True),
    types.StructField("pickup_datetime",  types.TimestampType(), True),
    types.StructField("dropoff_datetime",  types.TimestampType(), True),
    types.StructField("PULocationID",  types.LongType(), True),
    types.StructField("DOLocationID",  types.LongType(), True),
    types.StructField("trip_miles",  types.DoubleType(), True),
    types.StructField("trip_time",  types.LongType(), True),
    types.StructField("base_passenger_fare",  types.DoubleType(), True),
    types.StructField("tolls",  types.DoubleType(), True),
    types.StructField("bcf",  types.DoubleType(), True),
    types.StructField("sales_tax",  types.DoubleType(), True),
    types.StructField("congestion_surcharge",  types.DoubleType(), True),
    types.StructField("airport_fee",  types.DoubleType(), True),
    types.StructField("tips",  types.DoubleType(), True),
    types.StructField("driver_pay",  types.DoubleType(), True),
    types.StructField("shared_request_flag",  types.StringType(), True),
    types.StructField("shared_match_flag",  types.StringType(), True),
    types.StructField("access_a_ride_flag",  types.StringType(), True),
    types.StructField("wav_request_flag",  types.StringType(), True),
    types.StructField("wav_match_flag",  types.StringType(), True), 
])

In [28]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [29]:
df = df.repartition(24)

In [30]:
df.write.parquet('fhvhv/2021/01/')

22/11/22 13:35:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/22 13:35:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/22 13:35:21 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/22 13:35:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/22 13:35:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/22 13:35:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [31]:
df = spark.read.parquet('fhvhv/2021/01/')

In [48]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [56]:
from pyspark.sql import functions as F

In [61]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-07 06:43:22|2021-01-07 06:55:06|         142|         230|   null|
|           HV0005|              B02510|2021-01-01 16:01:26|2021-01-01 16:20:20|         133|          91|   null|
|           HV0003|              B02764|2021-01-01 00:23:13|2021-01-01 00:30:35|         147|         159|   null|
|           HV0003|              B02869|2021-01-06 11:43:12|2021-01-06 11:55:07|          79|         164|   null|
|           HV0003|              B02884|2021-01-04 15:35:32|2021-01-04 15:52:02|         174|          18|   null|
|           HV0003|              B02875|2021-01-04 13:42:15|2021-01-04 14:04:57|

In [63]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [65]:
crazy_stuff('B02884')

's/b44'

In [66]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [67]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-07|  2021-01-07|         142|         230|
|  e/9ce| 2021-01-01|  2021-01-01|         133|          91|
|  e/acc| 2021-01-01|  2021-01-01|         147|         159|
|  e/b35| 2021-01-06|  2021-01-06|          79|         164|
|  s/b44| 2021-01-04|  2021-01-04|         174|          18|
|  e/b3b| 2021-01-04|  2021-01-04|         201|         180|
|  e/9ce| 2021-01-04|  2021-01-04|         230|         142|
|  e/b38| 2021-01-03|  2021-01-03|         132|          72|
|  s/af0| 2021-01-01|  2021-01-01|         188|          61|
|  e/9ce| 2021-01-04|  2021-01-04|          97|         189|
|  e/acc| 2021-01-01|  2021-01-01|         174|         235|
|  a/b37| 2021-01-05|  2021-01-05|          35|          76|
|  e/9ce| 2021-01-06|  2021-01-06|          35|          39|
|  e/b42| 2021-01-04|  2

In [55]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')


[Row(pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 13), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 30, 35), PULocationID=147, DOLocationID=159),
 Row(pickup_datetime=datetime.datetime(2021, 1, 6, 11, 43, 12), dropoff_datetime=datetime.datetime(2021, 1, 6, 11, 55, 7), PULocationID=79, DOLocationID=164),
 Row(pickup_datetime=datetime.datetime(2021, 1, 4, 15, 35, 32), dropoff_datetime=datetime.datetime(2021, 1, 4, 15, 52, 2), PULocationID=174, DOLocationID=18),
 Row(pickup_datetime=datetime.datetime(2021, 1, 4, 13, 42, 15), dropoff_datetime=datetime.datetime(2021, 1, 4, 14, 4, 57), PULocationID=201, DOLocationID=180),
 Row(pickup_datetime=datetime.datetime(2021, 1, 3, 18, 42, 3), dropoff_datetime=datetime.datetime(2021, 1, 3, 19, 12, 22), PULocationID=132, DOLocationID=72)]

In [50]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag

HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,

HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,

HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,

HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,

HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,

HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,

HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,

HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,

HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,

